In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Import các thư viện thông dụng

In [ ]:
file_path = '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'
train_set = pd.read_csv(file_path)
predict_path = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
predict_set = pd.read_csv(predict_path)

Đọc dữ liệu cho 2 tập train và predict

In [ ]:
train_set.head()

In [ ]:
predict_set.head()

**B1: Xử lý dữ liệu trước khi đưa vào mô hình**


In [ ]:
train_set.shape

In [ ]:
train_set.isnull().sum()

In [ ]:
# Loại bỏ các hàng có output(sii) là NaN không có giá trị trong việc dự đoán
train_set = train_set.dropna(subset = 'sii')

In [ ]:
# Loại bỏ các giá trị NaN còn lại bằng giá trị mode của cột đó
for col in train_set:
    train_set[col] = train_set[col].fillna(train_set[col].mode()[0])
for col in predict_set:
    predict_set[col] = predict_set[col].fillna(predict_set[col].mode()[0])

In [ ]:
# Xử lý dữ liệu không hợp lệ, noise
# Bỏ qua

In [ ]:
# Chia tập train ra thành X_train và y_train
# Cắt bớt cột id của tập train để đưa vào mô hình
X_train = train_set.drop(['id','sii'], axis = 1).copy()
y_train = train_set['sii'].astype('int64').copy()

In [ ]:
# Cắt bớt cột id của tập test để đưa vào mô hình dự đoán
X_predict = predict_set.drop('id',axis = 1).copy()

In [ ]:
# Trong tập test, số lượng feature ít hơn tập train
# Có thể xóa bớt feature trong tập train hoặc thêm feature vào tập test
# Ở đây thêm vào feature cho đủ
for col in X_train:
    if col not in X_predict.columns:
        X_predict[col] = np.nan
        # Gán các giá trị cho cột feature bằng mode của tập train
        X_predict[col] = X_predict[col].fillna(X_train[col].mode())

# Sắp xếp lại tập test cho giống với tập train
X_predict = X_predict[X_train.columns]


In [ ]:
X_train.shape

In [ ]:
X_predict.shape

In [ ]:
y_train.shape

In [ ]:
y_train.dtype

**B2: Mã hóa dữ liệu**

Ta phải mã hóa 2 tập X_train và X_predict để có thể đưa vào mô hình có sẵn

In [ ]:
# Mã hóa dữ liệu
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 1 ma trận sử dụng để map giá trị X_predict và X_train
encoders = {}
#X_train
for column in X_train:
    encoder = LabelEncoder()
    X_train[column] = encoder.fit_transform(X_train[column])
    encoders[column] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
#X_predict
for column in X_predict:
    if column in encoders:
        X_predict[column] = X_predict[column].map(encoders[column]).fillna(X_train[column].mode()[0]).astype('int64')

In [ ]:
X_train.head()

In [ ]:
X_predict.head()

**B3: Lựa chọn mô hình và huấn luyện**

Với bài toán multiclassification, các mô hình có thể lựa chọn là:


* Logistic Regression (Multinomial)
* Decision Trees
* Random Forests
* k-Nearest Neighbors (k-NN)
* SVM
* Naive-Bayes

Cùng với đó là các mô hình nâng cao:
* XGboost: Kết hợp nhiều mô hình yếu để tạo ra 1 mô hình mạnh hơn, đây là thuật toán thường được sử dụng trong các competition nhờ độ hữu ích của nó
* Neural network: MLP hoặc mô hình deep learning có sẵn của thư viện tensorflow hoặc Pytorch

Ở đây em sẽ thử nghiệm 2 mô hình XGBoost và mạng NN của tensorflow

In [ ]:
# Import thư viện mô hình
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Chia tập train thành tập huấn luyện và kiểm tra

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Chuyển dữ liệu thành DMatrix của XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
# Tham số cho mô hình
params = {
    # Sử dụng hàm soft-max để dự đoán trực tiếp label
    'objective':'multi:softmax',
    # Số lượng label cần dự đoán, ở đây là 4
    'num_class':len(set(y_train)),
    #######################################
    # Loại booster sử dụng: decision tree
    'booster':'gbtree',

    ######################################
    # Thử nghiệm các tham số này để cải thiện độ chính xác
    # Learning rate
    'learning_rate':0.1,
    # Số lượng và độ sâu tối đa mỗi cây
    #'n_estimators':100,
    'max_depth':6,
    ######################################
    # Sử dụng Multi-class Logarithmic Loss làm metric đánh giá
    'eval_metric':'mlogloss'}


In [ ]:
evals_result = {}
model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dtrain, 'train'), (dtest, 'test')], evals_result=evals_result, early_stopping_rounds=10)

In [ ]:
# Vẽ biểu đồ hội tụ - Loss của training và test set
epochs = len(evals_result['train']['mlogloss'])
x_axis = range(0, epochs)

# Vẽ loss của training và test set
plt.figure(figsize=(10, 6))
plt.plot(x_axis, evals_result['train']['mlogloss'], label='Train')
plt.plot(x_axis, evals_result['test']['mlogloss'], label='Test')
plt.legend()
plt.xlabel('Number of Boosting Rounds')
plt.ylabel('Log Loss')
plt.title('XGBoost Training and Test Loss over Epochs')
plt.show()

In [ ]:
# Đánh giá mô hình
# Bỏ qua

**B4: Dự đoán kết quả**

In [ ]:
# Dự đoán
dPredict = xgb.DMatrix(X_predict)
predict = model.predict(dPredict).astype('int32')

In [ ]:
predict

**In ra file output**

In [ ]:
# In ra file output
import os
file_path = '/kaggle/working/submission.csv'
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"File {file_path} đã được xóa.")
else:
    print(f"File {file_path} không tồn tại.")

In [ ]:
pf_pred = pd.DataFrame(columns=['id', 'sii'])
pf_pred['id'] = predict_set['id']
pf_pred['sii'] = predict
pf_pred.head()
# set an index to avoid a 'blank' row number column and write to a file named 'submission.csv'
pf_pred.to_csv('submission.csv',index=False)

In [ ]:
# Kiểm tra dữ liệu file output
file_p = '/kaggle/working/submission.csv'
output = pd.read_csv(file_p)
output.head()

In [ ]:
# Các cách cải tiến dự đoán:
# Feature selection
# Tối ưu hóa tham số
# Thử nghiệm các mô hình khác
# Kết hợp nhiều mô hình, XGBoost